In [1]:
import torch
import torch.nn as nn
import torch.optim as optim


import numpy as np 
from AE import AE
from PointCloudDataset import PointCloudDataset

In [2]:
path_to_train = "data/shape_net_core_uniform_samples_2048/train.txt"
path_to_val   = "data/shape_net_core_uniform_samples_2048/val.txt"
path_to_test  = "data/shape_net_core_uniform_samples_2048/test.txt"

In [3]:
#Load Train, Val, Test Data

trainset = PointCloudDataset(path_to_data = path_to_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

valset = PointCloudDataset(path_to_data = path_to_val)
valloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = PointCloudDataset(path_to_data = path_to_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=True, num_workers=2)

In [4]:
batch_size = 512
epochs = 20
learning_rate = 1e-3

In [8]:
#  use gpu if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# create a model from `AE` autoencoder class
# load it to the specified device, either gpu or cpu
model = AE(input_shape=786432).to(device)

# create an optimizer object
# Adam optimizer with learning rate 1e-3
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# mean-squared error loss
criterion = nn.MSELoss()

In [10]:
for epoch in range(epochs):
    loss = 0
    for batch_features, _ in trainloader:
        # reshape mini-batch data to [N, 784] matrix
        # load it to the active device
        batch_features = batch_features.view(-1, 786432).to(device)
        batch_features = batch_features.float()
        
        # reset the gradients back to zero
        # PyTorch accumulates gradients on subsequent backward passes
        optimizer.zero_grad()
        
        # compute reconstructions
        outputs = model(batch_features)
        
        # compute training reconstruction loss
        train_loss = criterion(outputs, batch_features)
        
        # compute accumulated gradients
        train_loss.backward()
        
        # perform parameter update based on current gradients
        optimizer.step()
        
        # add the mini-batch training loss to epoch loss
        loss += train_loss.item()
    
    # compute the epoch training loss
    loss = loss / len(trainloader)
    
    # display the epoch training loss
    print("epoch : {}/{}, recon loss = {:.8f}".format(epoch + 1, epochs, loss))

RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/Users/oswald/opt/anaconda3/lib/python3.8/site-packages/torch/utils/data/_utils/worker.py", line 202, in _worker_loop
    data = fetcher.fetch(index)
  File "/Users/oswald/opt/anaconda3/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 47, in fetch
    return self.collate_fn(data)
  File "/Users/oswald/opt/anaconda3/lib/python3.8/site-packages/torch/utils/data/_utils/collate.py", line 83, in default_collate
    return [default_collate(samples) for samples in transposed]
  File "/Users/oswald/opt/anaconda3/lib/python3.8/site-packages/torch/utils/data/_utils/collate.py", line 83, in <listcomp>
    return [default_collate(samples) for samples in transposed]
  File "/Users/oswald/opt/anaconda3/lib/python3.8/site-packages/torch/utils/data/_utils/collate.py", line 63, in default_collate
    return default_collate([torch.as_tensor(b) for b in batch])
  File "